# JAMS-Test: evaluation of the jamifier

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import glob

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "choco"))


# Choose plotting style
# sns.set_style("whitegrid")
# sns.set_style("darkgrid")

In [3]:
def preprocess_partition_test_df(partition_test_path:str):

    partition_test_df = pd.read_csv(partition_test_path)
    # Append an extra column with the name of the outer partition
    partition_dir = os.path.dirname(os.path.dirname(partition_test_df["gold"][0]))
    if os.path.basename(partition_dir) == "test":  # still need to go one up
        partition_dir = os.path.dirname(partition_dir)
    partition_name = os.path.basename(partition_dir)
    partition_test_df["partition"] = partition_name

    return partition_test_df


def describe_partitions(partition_res_df:pd.DataFrame, dec=4):

    partition_means = partition_res_df.groupby("partition").mean().round(dec)
    partition_stds = partition_res_df.groupby("partition").std().round(dec)
    # String representation of the combined mean ± std partition-wise
    partition_str = partition_means.astype(str) \
        + " ± " + partition_stds.astype(str)
    partition_str = partition_str.replace("nan ± nan", "-")

    return partition_means, partition_stds, partition_str


## Processing a single evaluation

In [4]:
test_sample = "../partitions/isophonics/test/test_results_agg.csv"

test_sample_df = pd.read_csv(test_sample)

test_sample_df

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,gold,jamified,type
0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_164_s...,../partitions/isophonics/choco/jams/isophonics...,audio
1,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_200_s...,../partitions/isophonics/choco/jams/isophonics...,audio
2,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_73_si...,../partitions/isophonics/choco/jams/isophonics...,audio
3,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,../partitions/isophonics/test/isophonics_163_s...,../partitions/isophonics/choco/jams/isophonics...,audio


## Aggregation of evaluations

In [5]:
test_results = glob.glob("../partitions/*/test/**/test_results_agg.csv", recursive=True)
print(f"Found {len(test_results)} test result CSV files from partitions")

test_results

Found 19 test result CSV files from partitions


['../partitions/weimar/test/test_results_agg.csv',
 '../partitions/ireal-pro/test/playlists/test_results_agg.csv',
 '../partitions/rwc-pop/test/test_results_agg.csv',
 '../partitions/isophonics/test/test_results_agg.csv',
 '../partitions/uspop2002/test/test_results_agg.csv',
 '../partitions/when-in-rome/test/test_results_agg.csv',
 '../partitions/real-book/test/test_results_agg.csv',
 '../partitions/wikifonia/test/test_results_agg.csv',
 '../partitions/jazz-corpus/test/test_results_agg.csv',
 '../partitions/robbie-williams/test/test_results_agg.csv',
 '../partitions/nottingham/test/test_results_agg.csv',
 '../partitions/chordify/test/test_results_agg.csv',
 '../partitions/rock-corpus/test/test_results_agg.csv',
 '../partitions/schubert-winterreise/test/score/test_results_agg.csv',
 '../partitions/schubert-winterreise/test/audio/test_results_agg.csv',
 '../partitions/jaah/test/test_results_agg.csv',
 '../partitions/billboard/test/test_results_agg.csv',
 '../partitions/mozart-piano-sonat

In [6]:
partition_test_dfs = [preprocess_partition_test_df(ptest_path) for ptest_path in test_results]

In [7]:
all_partitions = pd.concat(partition_test_dfs)
# Recompute indexes and drop JAMS-file columns
all_partitions = all_partitions.reset_index(drop=True)
all_partitions = all_partitions.drop(["gold", "jamified"], axis=1)

In [8]:
meta_cols = ["meta_cov", "meta_acc", "iden_cov", "iden_acc", "partition"]
anns_cols = [c for c in all_partitions.columns if c not in meta_cols] + ["partition"]

all_partitions_meta = all_partitions[meta_cols]
all_partitions_anns = all_partitions[anns_cols]

all_partitions_anns_score = all_partitions_anns[all_partitions_anns["type"]=="score"]
all_partitions_anns_audio = all_partitions_anns[all_partitions_anns["type"]=="audio"]

In [9]:
all_partitions

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,type,partition
0,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
1,1.000,0.951389,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
2,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
3,1.000,1.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,audio,weimar
4,1.000,1.000000,NaN,NaN,0.0,1.0,1.0,1.1,0.0,0.0,0.8,1.0,1.0,1.0,0.0,0.000000,score,ireal-pro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.875,1.000000,1.0,1.0,1.0,0.0,0.0,0.0,53.0,0.5,1.0,0.5,0.0,0.0,0.4,0.590476,score,mozart-piano-sonatas
68,1.000,1.000000,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.8,1.0,1.0,0.2,0.0,0.000000,score,biab-internet-corpus
69,1.000,0.823171,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,score,biab-internet-corpus
70,0.800,0.888889,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.000000,score,biab-internet-corpus


In [10]:
partitions = list(all_partitions.partition.unique())

print(f"Found {len(partitions)} partitions: {partitions}")

Found 18 partitions: ['weimar', 'ireal-pro', 'rwc-pop', 'isophonics', 'uspop2002', 'when-in-rome', 'real-book', 'wikifonia', 'jazz-corpus', 'robbie-williams', 'nottingham', 'chordify', 'rock-corpus', 'schubert-winterreise', 'jaah', 'billboard', 'mozart-piano-sonatas', 'biab-internet-corpus']


### Check single partition

In [11]:
partition_name = "schubert-winterreise"

all_partitions[all_partitions["partition"]==partition_name]

,meta_cov,meta_acc,iden_cov,iden_acc,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value,type,partition
52,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,score,schubert-winterreise
53,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,score,schubert-winterreise
54,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,audio,schubert-winterreise
55,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,audio,schubert-winterreise


### Metadata results

In [12]:
_, _, meta_res_str = describe_partitions(all_partitions_meta)

In [13]:
meta_res_str

,meta_cov,meta_acc,iden_cov,iden_acc
partition,,,,
biab-internet-corpus,0.95 ± 0.1,0.9243 ± 0.0835,-,-
billboard,1.0 ± 0.0,1.0 ± 0.0,-,-
chordify,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0
ireal-pro,1.0 ± 0.0,1.0 ± 0.0,-,-
isophonics,1.0 ± 0.0,1.0 ± 0.0,-,-
jaah,0.8036 ± 0.0595,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0
jazz-corpus,1.0 ± 0.0,1.0 ± 0.0,-,-
mozart-piano-sonatas,0.875 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0
nottingham,1.0 ± 0.0,1.0 ± 0.0,-,-


### Annotation results: score

In [14]:
_, _, ann_res_score_str = describe_partitions(all_partitions_anns_score)

In [15]:
ann_res_score_str

,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value
partition,,,,,,,,,,,,
biab-internet-corpus,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.95 ± 0.1,1.0 ± 0.0,1.0 ± 0.0,0.05 ± 0.1,0.0 ± 0.0,0.0 ± 0.0
ireal-pro,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.1 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.2 ± 0.4,1.0 ± 0.0,1.0 ± 0.0,1.075 ± 0.05,0.0 ± 0.0,0.0 ± 0.0
jazz-corpus,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
mozart-piano-sonatas,0.5 ± 0.5774,0.0 ± 0.0,0.5 ± 0.5774,62.55 ± 125.0333,139.75 ± 83.7511,0.25 ± 0.2887,0.85 ± 0.3,0.875 ± 0.25,0.75 ± 0.5,0.25 ± 0.5,0.15 ± 0.1915,0.1476 ± 0.2952
nottingham,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.75 ± 0.2517,1.0 ± 0.0,1.0 ± 0.0,0.85 ± 0.5972,0.0 ± 0.0,0.0 ± 0.0
real-book,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
rock-corpus,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
schubert-winterreise,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
when-in-rome,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0


### Annotation results: audio

In [16]:
_, _, ann_res_audio_str = describe_partitions(all_partitions_anns_audio)

In [17]:
ann_res_audio_str


,key_cov_time,key_cov_duration,key_cov_value,key_err_time,key_err_duration,key_err_value,chord_cov_time,chord_cov_duration,chord_cov_value,chord_err_time,chord_err_duration,chord_err_value
partition,,,,,,,,,,,,
billboard,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
chordify,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
isophonics,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
jaah,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.95 ± 0.1,0.95 ± 0.1,1.0 ± 0.0,0.064 ± 0.128,0.064 ± 0.128,0.0 ± 0.0
robbie-williams,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
rwc-pop,-,-,-,-,-,-,1.0 ± 0.0,0.5333 ± 0.4522,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
schubert-winterreise,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
uspop2002,-,-,-,-,-,-,1.0 ± 0.0,0.3 ± 0.2582,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
weimar,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
